In [1]:
import imageio
import numpy as np
import pandas as pd
from skimage import transform
from IPython import display
from matplotlib import pyplot as plt
from src import camvid
from src import tiramisu
from src.utils import heatmap
from src.wrappers import MonteCarlo
%matplotlib inline
plt.style.use('dark_background')

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
mapping = pd.read_table('11_class.txt', sep=r'\s+', names=['og', 'new'], index_col='og')['new'].to_dict()
camvid11 = camvid.CamVid(mapping=mapping)

In [3]:
video_file = 'camvid/0005VD.mp4'
video_file

'camvid/0005VD.mp4'

In [4]:
out_file = video_file.split('.')[0] + '_pred.mp4'
out_file

'camvid/0005VD_pred.mp4'

In [5]:
reader = imageio.get_reader(video_file)

In [6]:
og_shape = reader.get_data(0).shape
og_shape

(720, 960, 3)

In [7]:
image_shape = (352, 480, 3)
image_shape

(352, 480, 3)

In [8]:
# build the model for the image shape and number of labels
model = tiramisu.build_epistemic_tiramisu(image_shape, camvid11.n)
model.layers[1].load_weights('models/Tiramisu103-CamVid11-fine.h5')

/Users/jameskauten/Projects/neural-semantic-segmentation/src/tiramisu.py:326: UserWarning: Output "entropy" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "entropy" during training.
  'mean': metrics_for_segmentation(num_classes, label_names, class_weights)


In [9]:
def predict(frame) -> tuple:
    """
    Return post-processed predictions for the given frame.

    Args:
        frame: the frame to predict

    Returns:
        a tuple of for NumPy tensors with RGB data:
        - the unmapped RGB predicted mean values from the model
        - the meatmap RGB values of the model variance

    """
    # predict mean values and variance
    X = frame[None, ...]
    u, s2 = model.predict(X)
    # normalize the uncertainty
    s2 = plt.Normalize()(s2)
    # return X values, unmapped u values, and heatmap of s2
    return camvid11.unmap(u)[0], heatmap(s2, 'afmhot')[0]

In [11]:
writer = imageio.get_writer(out_file)


for frame in reader:
    # resize the image to the acceptable size for the
    # model
    X = transform.resize(frame, image_shape,
        anti_aliasing=False,
        mode='symmetric',
        clip=False,
        preserve_range=True,
    )
    # predict mean and variance of the frame
    u, s2 = predict(X)
    h, w, c = image_shape
    # convert the three images into a singular image
    image = np.zeros((h, w * 3, c), dtype='uint8')
    image[:, 0:w, :] = X
    image[:, w:2*w, :] = u
    image[:, 2*w:3*w, :] = s2
    # save the image to the stream
    writer.append_data(image)


writer.close()